In [1]:
from sage.all import *

In [1]:
R = PolynomialRing(ZZ, 'x_A, x_B, x_C, y_A, y_B, y_C, a, b'.split(', '))
x_A, x_B, x_C, y_A, y_B, y_C, a, b = R.gens()

beta_tilde, gamma_tilde, alpha_tilde, tau_tilde = var('β̃ γ̃ α̃ τ̃')

alpha = (y_B - y_A) / (x_B - x_A)
beta = (y_A + y_C - alpha * (2*x_A + x_B - alpha**2)) / (x_A + x_B + x_C - alpha**2)
gamma = (y_B - y_C) / (x_B - x_C)
tau = (y_A + y_B - gamma * (2*x_B + x_C - gamma**2)) / (x_A + x_B + x_C - gamma**2)

alpha_tilde = y_B - x_A
beta_tilde = (y_A + y_C) * (x_B - x_A)**3 - alpha_tilde * ((2*x_A + x_B) * (x_B - x_A)**2 - alpha_tilde**2)
gamma_tilde = y_B - y_C
tau_tilde = (y_A + y_B) * (x_B - x_C)**3 - gamma_tilde * ((2*x_B + x_C) * (x_B - x_C)**2 - gamma_tilde**2)
eta_tilde = x_B - x_A
mu_tilde = x_B - x_C

term = (beta_tilde**2 * (x_B - x_C)**2 + (((2*x_A - 2*x_C)*(x_B - x_C)**2 + gamma_tilde**2) * (x_B - x_A)**2 - alpha_tilde**2 * (x_B - x_C)**2) *
        ((x_A + x_B + x_C) * (x_B - x_A)**2 - alpha_tilde**2)**2) * ((x_A + x_B + x_C) * (x_B - x_A)**2 - gamma_tilde**2)**2
-tau_tilde**2 * ((x_A + x_B + x_C) * (x_B - x_A)**2 - alpha_tilde**2)**2 * (x_B - x_A)**2

I = R.ideal(y_A**2 - x_A**3 - a*x_A - b,
            y_B**2 - x_B**3 - a*x_B - b,
            y_C**2 - x_C**3 - a*x_C - b)

print(term in I)

False


https://hyperelliptic.org/EFD/g1p/auto-montgom.html

In [14]:
def mynumerator(x):
    if parent(x) == R:
        return x
    return numerator(x)

class fastfrac:
    def __init__(self,top,bot=1):
        if parent(top) == ZZ or parent(top) == R:
            self.top = R(top)
            self.bot = R(bot)
        elif top.__class__ == fastfrac:
            self.top = top.top
            self.bot = top.bot * bot
        else:
            self.top = R(numerator(top))
            self.bot = R(denominator(top)) * bot
    def reduce(self):
        return fastfrac(self.top / self.bot)
    def sreduce(self):
        return fastfrac(I.reduce(self.top),I.reduce(self.bot))
    def iszero(self):
        return self.top in I and not (self.bot in I)
    def isdoublingzero(self):
        return self.top in J and not (self.bot in J)
    def __add__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top + self.bot * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot + self.bot * other.top,self.bot * other.bot)
        return NotImplemented
    def __sub__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top - self.bot * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot - self.bot * other.top,self.bot * other.bot)
        return NotImplemented
    def __neg__(self):
        return fastfrac(-self.top,self.bot)
    def __mul__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.top,self.bot * other.bot)
        return NotImplemented
    def __rmul__(self,other):
        return self.__mul__(other)
    def __div__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top,self.bot * other)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot,self.bot * other.top)
        return NotImplemented
    __truediv__ = __div__
    def __pow__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top ^ other,self.bot ^ other)
        return NotImplemented

def isidentity(x):
    return x.iszero()

def isdoublingidentity(x):
    return x.isdoublingzero()

R.<ua,ub,ux1,uy1,ux2,uy2> = PolynomialRing(QQ,6,order='invlex')
I = R.ideal([
    mynumerator((ub*uy1^2)-(ux1^3+ua*ux1^2+ux1))
    , mynumerator((ub*uy2^2)-(ux2^3+ua*ux2^2+ux2))
])

J = I + R.ideal([0
                    , ux1-ux2
                    , uy1-uy2
                 ])

ua = fastfrac(ua)
ub = fastfrac(ub)
ux1 = fastfrac(ux1)
uy1 = fastfrac(uy1)
ux2 = fastfrac(ux2)
uy2 = fastfrac(uy2)

ux3 = ((ub*(uy2-uy1)^2/(ux2-ux1)^2-ua-ux1-ux2)).reduce()
uy3 = (((fastfrac(2)*ux1+ux2+ua)*(uy2-uy1)/(ux2-ux1)-ub*(uy2-uy1)^3/(ux2-ux1)^3-uy1)).reduce()
ux4 = ((ub*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))^2/(fastfrac(2)*ub*uy1)^2-ua-ux1-ux1)).reduce()
uy4 = (((fastfrac(2)*ux1+ux1+ua)*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))/(fastfrac(2)*ub*uy1)-ub*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))^3/(fastfrac(2)*ub*uy1)^3-uy1)).reduce()
a0 = fastfrac((ub))
a1 = fastfrac((fastfrac(0)))
a2 = fastfrac((ua))
a3 = fastfrac((fastfrac(0)))
a4 = fastfrac((fastfrac(1)))
a6 = fastfrac((fastfrac(0)))
wweierx1 = ((ux1)).reduce().sreduce()
wweiery1 = ((uy1)).reduce().sreduce()
print(isidentity(a0*(wweiery1^2)+a1*(wweierx1*wweiery1)+a3*wweiery1-(((wweierx1+a2)*wweierx1+a4)*wweierx1+a6)))
print(isidentity(ux1-(wweierx1)))
print(isidentity(uy1-(wweiery1)))
wweierx2 = ((ux2)).reduce().sreduce()
wweiery2 = ((uy2)).reduce().sreduce()
wweierx3 = ((ux3)).reduce().sreduce()
wweiery3 = ((uy3)).reduce().sreduce()
wweierx4 = ((ux4)).reduce().sreduce()
wweiery4 = ((uy4)).reduce().sreduce()
slope = ((wweiery2-wweiery1)/(wweierx2-wweierx1)).reduce().sreduce()
print(isidentity(a0*slope^2+a1*slope-a2-wweierx1-wweierx2-wweierx3))
print(isidentity(slope*(wweierx1-wweierx3)-wweiery1-a1*wweierx3-a3-wweiery3))
slope = ((fastfrac(3)*wweierx1^2+fastfrac(2)*a2*wweierx1+a4-a1*wweiery1)/(fastfrac(2)*a0*wweiery1+a1*wweierx1+a3)).reduce().sreduce()
print(isidentity(a0*slope^2+a1*slope-a2-wweierx1-wweierx1-wweierx4))
print(isidentity(slope*(wweierx1-wweierx4)-wweiery1-a1*wweierx4-a3-wweiery4))


True
True
True
True
True
True
True


In [51]:
# Polynomring über Q
R.<a,b,x1,y1,x2,y2,x3,y3> = PolynomialRing(ZZ, 8, order='invlex')

# Ideal der elliptischen Kurve für alle drei Punkte
I = R.ideal([
    y1^2 - x1^3 - a*x1 - b,
    y2^2 - x2^3 - a*x2 - b,
    y3^2 - x3^3 - a*x3 - b
])

# Symbolische Addition gemäß fixer Formel
def add(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    x3 = (dy^2)/(dx^2) - x1 - x2
    y3 = ((2*x1 + x2)*dy)/(dx) - (dy^3)/(dx^3) - y1
    return x3, y3

# (P + Q) + R
xPQ, yPQ = add(x1, y1, x2, y2)
xPQR1, yPQR1 = add(xPQ, yPQ, x3, y3)

# P + (Q + R)
xQR, yQR = add(x2, y2, x3, y3)
xPQR2, yPQR2 = add(x1, y1, xQR, yQR)


print((xPQR1 - xPQR2) in I)  # True erwartet
print((yPQR1 - yPQR2) in I)  # True erwartet


False
False


In [42]:
R.<a,b,x1,y1,x2,y2,x3,y3> = PolynomialRing(QQ, 8, order='invlex')

# Kurvengleichung
I = R.ideal([
    y1 ^ 2 - x1 ^ 3 - a * x1 - b,
    y2 ^ 2 - x2 ^ 3 - a * x2 - b,
    y3 ^ 2 - x3 ^ 3 - a * x3 - b
])

# Entferne Sonderfälle (x1 = x2), (x2 = x3), (x3 = x1)
I = I.saturation(R.ideal([x1 - x2]))[0]
I = I.saturation(R.ideal([x2 - x3]))[0]
I = I.saturation(R.ideal([x3 - x1]))[0]


# Addition wie gehabt
def add(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    x3 = (dy ^ 2) / (dx ^ 2) - x1 - x2
    y3 = ((2 * x1 + x2) * dy) / (dx) - (dy ^ 3) / (dx ^ 3) - y1
    return x3, y3


# (P + Q) + R
xPQ, yPQ = add(x1, y1, x2, y2)
xPQR1, yPQR1 = add(xPQ, yPQ, x3, y3)

# P + (Q + R)
xQR, yQR = add(x2, y2, x3, y3)
xPQR2, yPQR2 = add(x1, y1, xQR, yQR)

print(xPQ.reduce() in I)

# Vergleich modulo Ideal (ohne Sonderfälle)
print((xPQR1 - xPQR2) in I)  # sollte True sein
print((yPQR1 - yPQR2) in I)  # sollte True sein


True
False
False


In [31]:
R.<ua,ub,ux1,uy1,ux2,uy2> = PolynomialRing(QQ,6,order='invlex')

# Helper: sicherstellen, dass Zähler extrahiert wird, wenn nötig
def mynumerator(x):
    return x if parent(x) == R else numerator(x)


# Kurvengleichung (Weierstrass-Form)
I = R.ideal([
    mynumerator(uy1 ^ 2 - (ux1 ^ 3 + ua * ux1 + ub)),
    mynumerator(uy2 ^ 2 - (ux2 ^ 3 + ua * ux2 + ub))
])

# Für Doubling-Identität: gleiche Punkte
J = I + R.ideal([
    ux1 - ux2,
    uy1 - uy2
])


# Rationalfunktionen definieren
def reduce(x):
    return R(x.numerator() / x.denominator())


def sreduce(x):
    return I.reduce(x.numerator()) / I.reduce(x.denominator())


# Identitätstest für Additionsformeln
def isidentity(x):
    return x in I


def isdoublingidentity(x):
    return x in J


# Umrechnen in Weierstrass-Normalform-Koeffizienten
a0, a1, a2, a3 = 1, 0, 0, 0
a4, a6 = ua, ub

# Reduzierte Punkte
x1r = reduce(ux1)
y1r = reduce(uy1)
x2r = reduce(ux2)
y2r = reduce(uy2)

# Berechne P + Q
dx = x2r - x1r
dy = y2r - y1r
x3 = reduce(dy ^ 2 / dx ^ 2 - x1r - x2r)
y3 = reduce(((2 * x1r + x2r) * dy / dx - dy ^ 3 / dx ^ 3 - y1r))

# Berechne 2P
num = 3 * x1r ^ 2 + ua
den = 2 * y1r
x4 = reduce(num ^ 2 / den ^ 2 - 2 * x1r)
y4 = reduce(((2 * x1r + x1r) * num / den - num ^ 3 / den ^ 3 - y1r))

# Nochmals idealreduziert
x1s = sreduce(x1r)
y1s = sreduce(y1r)
x2s = sreduce(x2r)
y2s = sreduce(y2r)
x3s = sreduce(x3)
y3s = sreduce(y3)
x4s = sreduce(x4)
y4s = sreduce(y4)

# Tests: Punkt liegt auf Kurve?
print(isidentity(a0 * y1s ^ 2 + a1 * x1s * y1s + a3 * y1s - (((x1s + a2) * x1s + a4) * x1s + a6)))
print(isidentity(ux1 - x1s))
print(isidentity(uy1 - y1s))

# Addition: Teste ob x3,y3 korrekt sind
slope = reduce((y2s - y1s) / (x2s - x1s))
slope = sreduce(slope)
print(isidentity(a0 * slope ^ 2 + a1 * slope - a2 - x1s - x2s - x3s))
print(isidentity(slope * (x1s - x3s) - y1s - a1 * x3s - a3 - y3s))

# Doubling: Teste ob x4,y4 korrekt sind
num = 3 * x1s ^ 2 + 2 * a2 * x1s + a4 - a1 * y1s
den = 2 * a0 * y1s + a1 * x1s + a3
slope2 = reduce(num / den)
slope2 = sreduce(slope2)
print(isidentity(a0 * slope2 ^ 2 + a1 * slope2 - a2 - 2 * x1s - x4s))
print(isidentity(slope2 * (x1s - x4s) - y1s - a1 * x4s - a3 - y4s))


TypeError: fraction must have unit denominator